In [36]:
from typing import List
import sympy as sym
from spatialmath import SE3
from spatialmath.base import *
import numpy as np
from numpy import sin, cos
from roboticstoolbox import ETS as ET
import roboticstoolbox as rtb
import matplotlib.pyplot as plt
from spatialmath import SE3

np.set_printoptions(precision=4, suppress=True, sign=" ")

In [54]:
def fkins(q5,q6, unit:str='rad'):
    pitch2yaw = 0.0092
    if unit == 'deg':
        q5 = q5*np.pi/180
        q6 = q6*np.pi/180

    alpha = 90 * np.pi/180
    #fmt:off
    transf = [ \
    [-sin(q5)*sin(q6)*cos(alpha)+cos(q5)*cos(q6), -sin(q5)*cos(alpha)*cos(q6)-sin(q6)*cos(q5),sin(alpha)*sin(q5),pitch2yaw*cos(q5)],\
    [ sin(q5)*cos(q6)+sin(q6)*cos(alpha)*cos(q5),-sin(q5)*sin(q6)+cos(alpha)*cos(q5)*cos(q6),-sin(alpha)*cos(q5),pitch2yaw*sin(q5)],\
    [ sin(alpha)*sin(q6), sin(alpha)*cos(q6), 0 , cos(alpha)],\
    [0,0,0,1]
    ]
    #fmt:on
    return np.array(transf)

def create_robot():
    pitch2yaw = 0.0092
    E = ET.rz() * ET.tx(pitch2yaw) * ET.rx(90, "deg") * ET.rz() *  ET.tx(0.1) * ET.tz(0.1) 
    robot = rtb.ERobot(E, name="test")
    return robot

def rtb_fkins(q5,q6,unit:str='rad'):
    pitch2yaw = 0.0092
    E = ET.rz() * ET.tx(pitch2yaw) * ET.rx(90, "deg") * ET.rz() *  ET.tx(0.1) * ET.tz(0.1)
    robot = rtb.ERobot(E, name="test")
    return robot.fkine([q5,q6],unit=unit) 

In [56]:
# My implementation
print("my implementation")
print(fkins(30,30,'deg'))
print("robot toolbox implementation")
robot_sol = rtb_fkins(30,30,'deg').data[0]
print(robot_sol)
print("Difference between implementations")
print(fkins(30,30,'deg')-robot_sol)

my implementation
[[ 0.75   -0.433   0.5     0.008 ]
 [ 0.433  -0.25   -0.866   0.0046]
 [ 0.5     0.866   0.      0.    ]
 [ 0.      0.      0.      1.    ]]
robot toolbox implementation
[[ 0.75   -0.433   0.5     0.133 ]
 [ 0.433  -0.25   -0.866  -0.0387]
 [ 0.5     0.866   0.      0.05  ]
 [ 0.      0.      0.      1.    ]]
Difference between implementations
[[ 0.      0.      0.     -0.125 ]
 [ 0.      0.      0.      0.0433]
 [ 0.      0.     -0.     -0.05  ]
 [ 0.      0.      0.      0.    ]]


In [51]:
robot_sol = rtb_fkins(30,45,'deg')
robot = create_robot()
s = robot.ikine_LM(robot_sol)
print(s)
q = s.q*180/np.pi
print(q)

IKsolution(q=array([ 0.5236,  0.7854]), success=True, reason=None, iterations=7, residual=7.762799069097578e-11)
[ 30.  45.]


## Mathematical derivation

In [13]:
#Transformation 1
theta = sym.symbols('theta5')
t1 = trotz(theta)
t1_sym = sym.Matrix(list(t1))
t1_sym

Matrix([
[cos(theta5), -sin(theta5), 0, 0],
[sin(theta5),  cos(theta5), 0, 0],
[          0,            0, 1, 0],
[          0,            0, 0, 1]])

In [8]:
#Transformation 2
pitch2yaw = 0.092
pitch2yaw = sym.symbols('pitch2yaw')
t2 = transl(pitch2yaw,0,0)
t2_sym = sym.Matrix(list(t2))
t2_sym

Matrix([
[1, 0, 0, pitch2yaw],
[0, 1, 0,         0],
[0, 0, 1,         0],
[0, 0, 0,         1]])

In [15]:
#Transformation 3
theta = sym.symbols('alpha')
t3 = trotx(theta)
t3_sym = sym.Matrix(list(t3))
t3_sym

Matrix([
[1,          0,           0, 0],
[0, cos(alpha), -sin(alpha), 0],
[0, sin(alpha),  cos(alpha), 0],
[0,          0,           0, 1]])

In [14]:
#Transformation 4
theta = sym.symbols('theta6')
t4 = trotz(theta)
t4_sym = sym.Matrix(list(t4))
t4_sym

Matrix([
[cos(theta6), -sin(theta6), 0, 0],
[sin(theta6),  cos(theta6), 0, 0],
[          0,            0, 1, 0],
[          0,            0, 0, 1]])

In [16]:
total = t1_sym*t2_sym*t3_sym*t4_sym
total


Matrix([
[-sin(theta5)*sin(theta6)*cos(alpha) + cos(theta5)*cos(theta6), -sin(theta5)*cos(alpha)*cos(theta6) - sin(theta6)*cos(theta5),  sin(alpha)*sin(theta5), pitch2yaw*cos(theta5)],
[ sin(theta5)*cos(theta6) + sin(theta6)*cos(alpha)*cos(theta5), -sin(theta5)*sin(theta6) + cos(alpha)*cos(theta5)*cos(theta6), -sin(alpha)*cos(theta5), pitch2yaw*sin(theta5)],
[                                       sin(alpha)*sin(theta6),                                        sin(alpha)*cos(theta6),              cos(alpha),                     0],
[                                                            0,                                                             0,                       0,                     1]])

In [17]:
sym.trigsimp(total)

Matrix([
[-sin(theta5)*sin(theta6)*cos(alpha) + cos(theta5)*cos(theta6), -sin(theta5)*cos(alpha)*cos(theta6) - sin(theta6)*cos(theta5),  sin(alpha)*sin(theta5), pitch2yaw*cos(theta5)],
[ sin(theta5)*cos(theta6) + sin(theta6)*cos(alpha)*cos(theta5), -sin(theta5)*sin(theta6) + cos(alpha)*cos(theta5)*cos(theta6), -sin(alpha)*cos(theta5), pitch2yaw*sin(theta5)],
[                                       sin(alpha)*sin(theta6),                                        sin(alpha)*cos(theta6),              cos(alpha),                     0],
[                                                            0,                                                             0,                       0,                     1]])